In [6]:
import pyrealsense2 as rs
import numpy as np
import cv2
import time
import matplotlib.pyplot as plt
import h5py
import os
import argparse
import multiprocessing as mp
import yaml
import warnings
from queue import LifoQueue
from threading import Thread

datadir = r'D:\DATA\JB\realsense'
name = 'front_right_JB043'
preview = True
save = False
master = '830112071475'
verbose = True

In [2]:
def enumerate_connected_devices(context):
    """
    Enumerate the connected Intel RealSense devices
    Parameters:
    -----------
    context           : rs.context()
                         The context created for using the realsense library
    Return:
    -----------
    connect_device : array
                       Array of enumerated devices which are connected to the PC
    """
    connect_device = []
    for d in context.devices:
        if d.get_info(rs.camera_info.name).lower() != 'platform camera':
            connect_device.append(d.get_info(rs.camera_info.serial_number))
    return connect_device

In [51]:
serials = enumerate_connected_devices(rs.context())
if verbose:
    print('Serials: ', serials)
assert(master in serials)
if os.path.isfile('serials.yaml'):
    with open('serials.yaml') as f:
        serial_dict = yaml.load(f, Loader=yaml.SafeLoader)
else:
    warnings.Warn('Need to create a lookup table for matching serial numbers with names.')
    serial_dict = {}
    for serial in serials:
        serial_dict[serial] = None


Serials:  ['817412070989', '830112071475', '817412071174', '832112070808']
{'830112071475': 'right_post', '817412070989': 'left_post', '832112070808': 'right_ant', '817412071174': 'left_ant'}


In [53]:
for key, item in serial_dict.items():
    print('%s: %s' %(item,key))

right_post: 830112071475
left_post: 817412070989
right_ant: 832112070808
left_ant: 817412071174


In [54]:
resolution_width = 640
resolution_height = 480
framerate=60

In [55]:
serial = '830112071475'

In [77]:
def print_extrinsic_intrinsic(serial):
    print('~~~ Serial ~~~')
    print(serial)
    config = rs.config()
    config.enable_stream(rs.stream.infrared, 1, resolution_width, resolution_height, rs.format.y8, framerate)
    config.enable_stream(rs.stream.infrared, 2, resolution_width, resolution_height, rs.format.y8, framerate)
    # config.enable_stream(rs.stream.depth, resolution_width, resolution_height, rs.format.z16, framerate)

    pipeline = rs.pipeline()
    config.enable_device(serial)
    pipeline_profile = pipeline.start(config) 
    
    profile = pipeline_profile.get_stream(rs.stream.infrared,1)
    intr = profile.as_video_stream_profile().get_intrinsics()
    
    
    K = np.zeros((3,3),dtype=np.float64)
    K[0,0] = intr.fx
    K[0,2] = intr.ppx
    K[1,1] = intr.fy
    K[1,2] = intr.ppy
    K[2,2] = 1
    print('~~~ Intrinsics ~~~')
    print(K)
    
    extrinsics = pipeline_profile.get_stream(rs.stream.infrared,1).as_video_stream_profile().get_extrinsics_to(pipeline_profile.get_stream(
        rs.stream.infrared,2))
    print('~~~ Extrinsics, left IR to right IR ~~~')
    print('Rotation: ', extrinsics.rotation)
    print('Translation: ', extrinsics.translation)
    
    pipeline.stop()
    config.disable_all_streams()

In [86]:
[print_extrinsic_intrinsic(serial) for serial in serials]

~~~ Serial ~~~
817412070989
~~~ Intrinsics ~~~
[[385.8394165    0.         317.54040527]
 [  0.         385.8394165  243.37915039]
 [  0.           0.           1.        ]]
~~~ Extrinsics, left IR to right IR ~~~
Rotation:  [1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]
Translation:  [-0.049900662153959274, 0.0, 0.0]
~~~ Serial ~~~
830112071475
~~~ Intrinsics ~~~
[[385.20562744   0.         320.95071411]
 [  0.         385.20562744 244.75238037]
 [  0.           0.           1.        ]]
~~~ Extrinsics, left IR to right IR ~~~
Rotation:  [1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]
Translation:  [-0.04994235932826996, 0.0, 0.0]
~~~ Serial ~~~
817412071174
~~~ Intrinsics ~~~
[[393.30001831   0.         315.61096191]
 [  0.         393.30001831 240.31312561]
 [  0.           0.           1.        ]]
~~~ Extrinsics, left IR to right IR ~~~
Rotation:  [1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]
Translation:  [-0.04998056963086128, 0.0, 0.0]
~~~ Serial ~~~
832112070808
~~~ Intrinsics ~~

[None, None, None, None]

rotation: [1, 0, 0, 0, 1, 0, 0, 0, 1]
translation: [-0.0499007, 0, 0]